# Learning Canonical Transformations to Conserved Quantities

## Neumann Hamiltonian

We consider the integrable Hamiltonian

$$
H = \frac{1}{4}\sum_{\alpha,\beta} J_{\alpha\beta}^2 + \frac{1}{2}\sum_\alpha k_\alpha x_\alpha^2
$$

with $J_\alpha\beta = x_\alpha p_\beta - x_\beta p_\alpha$ is the (generalized) angular momentum. The constants of motion are

$$
I_\alpha = x_\alpha^2 + \sum_{\beta\neq \alpha} \frac{J_{\alpha\beta}^2}{k_\alpha-k_\beta}
$$

for $k_\alpha$ all different. We have

$$
H = \frac{1}{2}\sum_\alpha k_\alpha I_\alpha
$$

And

$$
\sum_\alpha x_\alpha^2 = \sum_\alpha I_\alpha,
$$

so that motion is confined to the sphere $S^{N-1}$.

## Seeking Constants of Motion

We are going to seek a canonical transformation to variables $q_\alpha$, $\pi_\alpha$ in which the radial coordinates

$$
\rho_\alpha = \sqrt{q_\alpha^2 + \pi_\alpha^2}
$$

are conserved. This means that we express the transformed Hamiltonian as $H'(q,\pi)\equiv H(x(q,\pi), p(q,\pi))$ and then try to set

$$
q_\alpha \dot q_\alpha +\pi_\alpha \dot \pi_\alpha=q_\alpha \frac{\partial H'}{\partial \pi_\alpha}-\pi_\alpha \frac{\partial H'}{\partial q_\alpha}=0,
$$

which suggests the loss function

$$
\ell(q,\pi) = \sum_\alpha \left(q_\alpha \frac{\partial H'}{\partial \pi_\alpha}-\pi_\alpha \frac{\partial H'}{\partial q_\alpha}\right)^2,
$$

which could be evaluated e.g. on i.i.d. Gaussian variates. Compared to the trajectory approach:

1. We don't need to solve an ODE, but
2. We do need to differentiate to find the gradients.

In [1]:
import sys
sys.path.append("../")
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
import matplotlib.pyplot as plt

In [2]:
from hamiltonians import parameterized_neumann, parameterized_oscillator
from models import NICE, SymplecticExchange, SqueezeAndShift, Chain, MLP, IrrotationalMLP, LinearSymplecticTwoByTwo
from utils import visualize_chain_bijector_1d, extract_q_p, join_q_p, is_symplectic, run_eagerly

In [3]:
# tf.enable_eager_execution()
sess = tf.InteractiveSession()
tf.set_random_seed(0)

In [4]:
N = 3
ks = [0.9743002 , 0.15526697, 0.46813834] # Just to fix some values
ks_tensor = tf.constant(ks, dtype=tf.float32, name="ks")
# ks_tensor = tf.constant([1,1,1], dtype=tf.float32, name="ks")

## Form Transformation by Chaining Bijectors

The idea is to map _from_ trajectories _to_ the phase plane of action angle variables, imposing the condition that the distribution factorizes.

In [5]:
# Suppress the warning till they fix this:
# lib/python3.5/site-packages/tensorflow/python/util/tf_inspect.py:75: 
# DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
import warnings
warnings.filterwarnings("ignore")

In [6]:
DTYPE=tf.float32
NP_DTYPE=np.float32
settings = {
    'hamiltonian': parameterized_neumann(ks_tensor), 
    'phase_space_dim': 2*N, 
    'batch_size': 1024,    # Mini batch size 
    'stack_size': 2}        # How many sets of NICE / SymplecticExchange / LinearSymplecticTwoByTwo

Here we try modeling with:

1. `IrrotationalMLP`, which directly generates an irrotational vector field, without using autodifferentiation.

2. `LinearSymplecticTwoByTwo`, which learns linear symplectic transformations on each conjugate pair. 

There is freedom to choose different activations here. Could use traditional NN nonlinearities or simpler things like `tf.square`. Since our loss now depends on gradients, should chose a smooth activation.

We don't really need squeeze, as the linear transformation includes this, so we just use NICE.

Seem to be important to do `SymplecticExchange` (an even number of times).

In [7]:
bijectors = []
for _ in range(settings['stack_size']):
    bijectors.extend([SymplecticExchange(),
                      NICE(shift_model=IrrotationalMLP(activation=tf.nn.tanh, width=32)),
                      LinearSymplecticTwoByTwo()])
# bijectors = 2*[NICE(shift_model=IrrotationalMLP(activation=tf.nn.tanh)), 
#                       SymplecticExchange(), 
#                       LinearSymplecticTwoByTwo()]
model = Chain(bijectors)

## Form the Loss

In [8]:
with tf.name_scope("loss"):
    base_dist = tfd.MultivariateNormalDiag(loc=tf.zeros([settings['phase_space_dim']], DTYPE))
    z_samples = tf.expand_dims(base_dist.sample(settings['batch_size']), -1)
    Hprime = settings['hamiltonian'](model(z_samples))
    Hprime_grads = tf.gradients(Hprime, z_samples)[0]
    q, p = extract_q_p(z_samples)
    q_grads, p_grads = extract_q_p(Hprime_grads)
    loss = tf.reduce_mean(tf.square(q * p_grads - p * q_grads))

## Train

In [9]:
step = tf.get_variable("global_step", [], tf.int64, tf.zeros_initializer(), trainable=False)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step=step)

Add some summaries for tracking with TensorBoard

In [10]:
tf.summary.scalar("loss_function", tf.reshape(loss, []))

<tf.Tensor 'loss_function:0' shape=() dtype=string>

Monitor training with `tensorboard --logdir logging/`

In [ ]:
tf.contrib.training.train(train_op, logdir="../logging", save_checkpoint_secs=60)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ../logging/model.ckpt.
INFO:tensorflow:global_step/sec: 78.122
INFO:tensorflow:global_step/sec: 128.136
INFO:tensorflow:global_step/sec: 137.726
INFO:tensorflow:global_step/sec: 138.89
INFO:tensorflow:global_step/sec: 139.603
INFO:tensorflow:global_step/sec: 138.396
INFO:tensorflow:global_step/sec: 139.928
INFO:tensorflow:global_step/sec: 139.826
INFO:tensorflow:global_step/sec: 120.504
INFO:tensorflow:global_step/sec: 117.308
INFO:tensorflow:global_step/sec: 111.78
INFO:tensorflow:global_step/sec: 100.246
INFO:tensorflow:global_step/sec: 116.302
INFO:tensorflow:global_step/sec: 117.736
INFO:tensorflow:global_step/sec: 125.86
INFO:tensorflow:global_step/sec: 112.387
INFO:tensorflow:global_step/sec: 116.344
INFO:tensorflow:global_step/sec: 101.931
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:global_step/sec: 133.672
INFO:tensorflow:global_step/sec: 132.931
INFO:tensorflow:global_step/sec: 134.491
INFO:tensorflow:global_step/sec: 131.138
INFO:tensorflow:global_step/sec: 138.002
INFO:tensorflow:global_step/sec: 131.275
INFO:tensorflow:global_step/sec: 135.987
INFO:tensorflow:global_step/sec: 134.597
INFO:tensorflow:global_step/sec: 131.617
INFO:tensorflow:global_step/sec: 124.392
INFO:tensorflow:global_step/sec: 127.509
INFO:tensorflow:global_step/sec: 130.575
INFO:tensorflow:global_step/sec: 133.383
INFO:tensorflow:global_step/sec: 129.676
INFO:tensorflow:global_step/sec: 123.603
INFO:tensorflow:global_step/sec: 137.326
INFO:tensorflow:global_step/sec: 136.62
INFO:tensorflow:global_step/sec: 129.26
INFO:tensorflow:global_step/sec: 134.663
INFO:tensorflow:global_step/sec: 136.463
INFO:tensorflow:global_step/sec: 137.835
INFO:tensorflow:global_step/sec: 138.595
INFO:tensorflow:global_step/sec: 137.203
INFO:tensorflow:global_step/sec: 138.974
INFO:tensorflow:gl

## Look at the Learned Representation

Load the saved model. If not training, first evaluate all cells _except_ for those in the __Train__ section. 

In [ ]:
traindir = "../logging"
saver = tf.train.Saver()

In [ ]:
ckpt_state = tf.train.get_checkpoint_state(traindir)
saver.restore(sess, ckpt_state.model_checkpoint_path)

How do the trajectories look in the phase planes of the new variables? 

In [ ]:
def hamiltons_equations(H):
    def flow(phase_space_point, t):
        phase_space_point = tf.reshape(phase_space_point, [1,-1,1])
        H_grads = tf.gradients(H(phase_space_point), phase_space_point)[0]
        dHdq, dHdp = extract_q_p(H_grads)
        flow_vec = join_q_p(dHdp, -dHdq)
        return flow_vec[0,:,0]
    return flow

def hamiltonian_traj(H, init_state, time=100, steps=200, rtol=1e-04, atol=1e-6):
    t = tf.linspace(0.0, time, num=steps)
    tensor_state = tf.contrib.integrate.odeint(hamiltons_equations(H), init_state, t, rtol, atol)
    return tensor_state

In [ ]:
init_state = base_dist.sample(1)[0]
traj = sess.run(hamiltonian_traj(settings['hamiltonian'], init_state, time=20))
x1, y1, x2, y2, x3, y3 = traj.T

In [ ]:
trans_traj = sess.run(model.inverse(tf.expand_dims(traj, axis=-1)))
q1, p1, q2, p2, q3, p3 = trans_traj[:,:,0].T

In [ ]:
plt.figure(figsize=(20,20))

plt.subplot(1, 3, 1)
plt.plot(x1, y1)
plt.plot(q1, p1)
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')

plt.subplot(1, 3, 2)
plt.plot(x2, y2)
plt.plot(q2, p2)
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')

plt.subplot(1, 3, 3)
plt.plot(x3, y3)
plt.plot(q3, p3)
plt.xlim(-10, 10)
plt.ylim(-10, 10)
plt.gca().set_aspect('equal', adjustable='box')


In [ ]:
x = sess.run(z_samples)
samples = [x]
names = ["base_dist"]
for bijector in model.bijectors:
    x = sess.run(bijector(x))
    samples.append(x)
    names.append(bijector.name)
f, arr = plt.subplots(1, len(samples), figsize=(4 * (len(samples)), 4))
X0 = samples[0][:,:2,0]
for i, sample in enumerate(samples):
    X1 = sample[:,:2,0]
    idx = np.logical_and(X0[:, 0] < 0, X0[:, 1] < 0)
    arr[i].scatter(X1[idx, 0], X1[idx, 1], s=10, color='red')
    idx = np.logical_and(X0[:, 0] > 0, X0[:, 1] < 0)
    arr[i].scatter(X1[idx, 0], X1[idx, 1], s=10, color='green')
    idx = np.logical_and(X0[:, 0] < 0, X0[:, 1] > 0)
    arr[i].scatter(X1[idx, 0], X1[idx, 1], s=10, color='blue')
    idx = np.logical_and(X0[:, 0] > 0, X0[:, 1] > 0)
    arr[i].scatter(X1[idx, 0], X1[idx, 1], s=10, color='black')
    arr[i].set_xlim([-10, 10])
    arr[i].set_ylim([-10, 10])
    arr[i].set_title(names[i])